<a href="https://colab.research.google.com/github/stevejj4/Insurance-data-lifecycle/blob/main/Customers_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-bigquery
!pip install pyspark==3.1.2

from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd
from pyspark.sql import SparkSession


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 14.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880745 sha256=64d6e11063f1fb54ba14f133d5629404ee75aec63500ad78fd1ffdb3c8ee6454
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
# Authenticating and initializing BigQuery client
project_id = 'river-messenger-430112-e1'
client = bigquery.Client(project=project_id)

In [ ]:
# Querying to get interactions table
query_customers = """
SELECT * FROM `river-messenger-430112-e1.Insurance_data.customers`;
"""

In [ ]:
# Executing the query and converting to a pandas DataFrame
df_customers = client.query(query_customers).to_dataframe()

In [15]:
# Initializing SparkSession
spark = SparkSession.builder \
    .appName('BigQuerySparkApp') \
    .getOrCreate()

# Converting pandas DataFrame to Spark DataFrame
# Using iterrows() instead of iteritems() to iterate over DataFrame rows
spark_df_customers = spark.createDataFrame(df_customers.to_dict('records'))

# Show the schema and first few rows
spark_df_customers.printSchema()
#spark_df_interactions.show(5)

root
 |-- Age: long (nullable = true)
 |-- CustomerID: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Region: string (nullable = true)



In [16]:
spark_df_customers.show(5)

+---+----------+------+-----------------+
|Age|CustomerID|Gender|           Region|
+---+----------+------+-----------------+
| 18|        43|  Male|   Allisonborough|
| 18|       189|  Male|      Lake Lauren|
| 18|       373|  Male|      North Renee|
| 18|       411|  Male|West Lindsayville|
| 18|       420|  Male|    Chandlershire|
+---+----------+------+-----------------+
only showing top 5 rows



In [20]:
from pyspark.sql.functions import col, when, count, sum

# Calculate age distribution with percentages
age_distribution = spark_df_customers.select(
    when(col("age") < 25, "Under 25")
    .when((col("age") >= 25) & (col("age") <= 34), "25-34")
    .when((col("age") >= 35) & (col("age") <= 44), "35-44")
    .when((col("age") >= 45) & (col("age") <= 54), "45-54")
    .when((col("age") >= 55) & (col("age") <= 64), "55-64")
    .otherwise("65+")
    .alias("age_group")
).groupBy("age_group").agg(count("*").alias("count"))

# Calculate total count
total_count = spark_df_customers.count()

# Add percentage column
age_distribution_with_percentage = age_distribution.withColumn(
    "percentage", (col("count") / total_count) * 100
)

# Display the result
age_distribution_with_percentage.show()

+---------+-----+------------------+
|age_group|count|        percentage|
+---------+-----+------------------+
|    45-54|  379|18.912175648702593|
|    55-64|  394|19.660678642714572|
| Under 25|  277|13.822355289421157|
|    35-44|  378|18.862275449101794|
|    25-34|  355|17.714570858283434|
|      65+|  221|11.027944111776447|
+---------+-----+------------------+



In [22]:
from pyspark.sql.functions import col, count

# Assuming 'gender' is the column containing gender information
gender_distribution = spark_df_customers.groupBy("gender").agg(count("*").alias("count"))

# Calculate total count
total_count = spark_df_customers.count()

# Add percentage column
gender_distribution_with_percentage = gender_distribution.withColumn(
    "percentage", (col("count") / total_count) * 100
)

# Display the result
gender_distribution_with_percentage.show()

+------+-----+------------------+
|gender|count|        percentage|
+------+-----+------------------+
|Female|  962|48.003992015968066|
|  Male| 1042|51.996007984031934|
+------+-----+------------------+

